In [1]:
# Dependencies
import pandas as pd

In [2]:
# Source files
attendance_raw = 'Resources/national_park_attendance_Prev3Y.xlsx'
focus_parks_csv = 'Resources/califlo.csv'

# Read data and store in dataframe
attendance_data = pd.read_excel(attendance_raw)
focus_parks = pd.read_csv(focus_parks_csv)


In [4]:
attendance_data.head()

,Park,Code,Month,Visitors
0,Abraham Lincoln Birthplace NHP,ABLI,2020-01-01 00:00:00,6911.0
1,Acadia NP,ACAD,2020-01-01 00:00:00,12640.0
2,Adams NHP,ADAM,2020-01-01 00:00:00,300.0
3,African Burial Ground NM,AFBG,2020-01-01 00:00:00,4279.0
4,Agate Fossil Beds NM,AGFO,2020-01-01 00:00:00,107.0


In [3]:
focus_parks.head()

,National Park,Coordinates
0,Biscayne,"{'latitude': 25.65, 'longitude': -80.08}"
1,Channel Islands,"{'latitude': 34.01, 'longitude': -119.42}"
2,Death Valley,"{'latitude': 36.24, 'longitude': -116.82}"
3,Dry Tortugas,"{'latitude': 24.63, 'longitude': -82.87}"
4,Everglades,"{'latitude': 25.32, 'longitude': -80.93}"
